## DeepFool Algorithm

- **Authors:** Guilherme Magalhães, João Sousa, João Baptista
- **University:** Faculty of Sciences of University of Porto
- **Course:** Machine Learning II (CC3043)

For a given classifier and example, the algorithm is set to compute the minimal perturbation that is sufficient to change the estimated label. For the remainder of the section, the following notation will be used:

- $f$: a given classifier that outputs a vector with the probability distribution for the classification associated with its probability index.

- $x$: a given example.

- $X$: the domain of the examples.

- $T$: the domain of test examples available.

- $k$: a possible classification of the considered problem. Thus, the probability of the classification of an example $x$ to be $k$ is $f_k(x)$.

- $\hat{k}(x)$: the estimated classification of a given example. It is noted that $\hat{k}(x) = argmax_k( f_k(x) )$.

- $\hat{r}(x)$: the minimal perturbation for which $\hat{k}(x) \ne \hat{k}(x+\hat{r}(x))$.

The DeepFool algorithm only outputs the value of the minimal perturbation $\hat{r}(x)$ of a given example $x$.

Finally, the proposed formal definition for the robustness to adversarial examples of a given classifier is the expected value over the domain of examples for the norm of the minimal perturbation for an example divided by the norm of that same example. For practical purposes, the aforementioned expected value is approximated to the mean value for all examples in the available test domain of the classifier:

$\rho_{adv}(f) = 𝔼_X \frac{||\hat{r}(x)||_2}{||x||_2} ≈ \frac{1}{|T|} ∑_{x\in T} \frac{||\hat{r}(x)||_2}{||x||_2}$

### Necessary Imports

In [ ]:
import pickle
import pandas as pd
import numpy as np
import os
import librosa
import tensorflow as tf
import numpy as np
from copy import deepcopy
import keras
import gc
from tensorflow.keras import optimizers


### Saving and Loading Data

In [4]:
def save_pkl(data, path):
    try:
        with open(path, "wb") as saved_data:
            pickle.dump(data, saved_data)
        saved_data.close()
    except:
        print('Fail to save data')

def load_pkl(path):
    try:
        with open(path, "rb") as loaded_data:
            to_return = pickle.load(loaded_data)
        loaded_data.close()
        return to_return
    except:
        print('Fail to load data')
        return None

### Implementation of the Algorithm

In [ ]:
# gradient of class k with respect to the input
def get_gradient(model, x, k):
    x = tf.convert_to_tensor(x, dtype=tf.float32)

    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x)
        # model's output
        results = model(x)
        results_k = results[0, k]  # k-th class output for the first input in the batch
    gradients = tape.gradient(results_k, x)  

    del tape
    return gradients.numpy(), results

In [ ]:
# calculate robustness of an individual example 
def example_robustness(x, r):
    r_norm = np.sqrt(np.sum([np.linalg.norm(r_input)**2 for r_input in r]))
    x_norm = np.sqrt(np.sum([np.linalg.norm(x_input)**2 for x_input in x]))
    return r_norm / x_norm

In [ ]:
# model: the model to be used in the algorithm
# x0: the initial input without any perturbation
# eta: an overshoot value to be multiplied with each iteration's perturbation
# max_iter: maximum nummber of iterations  the algorithm is allowed to execute
def deepfool(model, x0, eta=0.01, max_iter=20):

    # obtain the initial estimated label
    f_x0 = model.predict(x0).flatten()
    print(f"f_x0 = {f_x0}")
    label_x0 = f_x0.argsort()[::-1][0]

    loop_i = 0
    xi = deepcopy(x0)
    label_xi = label_x0
    r = []

    # main loop
    while label_xi == label_x0 and loop_i < max_iter:
        w_l = [np.zeros(x_input.shape) for x_input in x0]
        f_l = 0
        fk_wk_min = np.inf
        grad_f_label_x0_on_xi, f_xi = get_gradient(model, xi, label_x0)

        for k in range(10): # k = 0, ..., 9 (possible classes in the problem considered for this project)
            if (k == label_x0):
                continue
            grad_f_k_on_xi, f_xi = get_gradient(model, xi, k)
            w_k = [g_f_k - g_f_label for g_f_k, g_f_label in zip(grad_f_k_on_xi, grad_f_label_x0_on_xi)]
            w_k_norm = np.sqrt(np.sum(np.fromiter([np.linalg.norm(w_k_input)**2 for w_k_input in w_k], dtype=np.float32)))
            f_k = f_xi[0,k] - f_xi[0,label_x0]
            fk_wk = np.linalg.norm(f_k) / (w_k_norm + 1e-3)
            if fk_wk < fk_wk_min:
                w_l, f_l = w_k, f_k
        
        w_l_squared_norm = np.sum(np.fromiter([np.linalg.norm(w_l_input)**2 for w_l_input in w_l], dtype=np.float32))
        f_l_norm = np.linalg.norm(f_l)
        ri_const = f_l_norm / (w_l_squared_norm + 1e-3)
        ri = [ri_const * w_l_input for w_l_input in w_l]
        r.append(ri)
        xi_new = [xi_item + (1+eta)*ri_item for xi_item, ri_item in zip(xi, ri)]
        xi = xi_new
        xi = np.array(xi)
        print(f"xi shape: {xi.shape}")
        label_xi = model.predict(xi).flatten()
        label_xi = label_xi.argsort()[::-1][0]
        loop_i += 1

    # main loop finished
    r_sum = [np.zeros(x_input.shape) for x_input in x0]
    for i in range(len(x0)):
        for r_i in r:
            r_sum[i] += r_i[i][0]

    # return the value of r(x), number of iterations performed, and the new label obtained by adding the perturbation to the input
    return r_sum, loop_i, label_xi


### Running DeepFool for the RNN-LSTM

Robustness for a classifier is practically defined as the mean value of the norm of the minimal perturbation required for an example, divided by the norm of the example itself.

Since the RNN developed in this project was evaluated using a 10-fold cross-validation approach, it was necessary to calculate the robustness for each model trained during the cross-validation process.

For each iteration, the test data used to assess robustness to adversarial examples was the same as the test data used to compute the cross-validation metrics in the notebook rnn-lstm.ipynb.

The final robustness results are presented as the mean and standard deviation of the values obtained across all cross-validation test datasets. These results can be found in rnn-lstm.ipynb.

In [34]:
df_data = load_pkl("data/features_rnn.pkl")

In [ ]:
for f in range(1, 10+1):

    # this list will hold the values ||r(x)|| / ||x|| initially mentioned on the notebook 
    robustness_values_rnn_fold = []

    # utilize only the data of the f-th fold
    X_fold = df_data[f]

    
    

    # load the model from the performance evaluation cross validation run that used the f-th fold as the test fold
    # the compile keyword argument was set to false such that the model can be explicitly re-compiled using the compile settings defined on LSTM.ipynb
    fold_model_rnn = keras.models.load_model(f"kfold_metrics_LSTM/model_fold{f}.keras", compile=False)
    fold_model_rnn.compile(
        optimizer=optimizers.Adam(learning_rate=0.001),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    # begin the run of each example in the f-th fold and the corresponding model on the DeepFool algorithm
    for i in range(len(X_fold)):
        print(f"Shape of X_fold[{i}]: {X_fold[i].shape}")  # Debugging
    
        example_input = np.expand_dims(X_fold[i], axis = 0)
        print(f"Shape of example_input: {example_input.shape}")

        # prepare the input for the implemented DeepFool function
        

        # run DeepFool
        # eta keyword argument was set to 10^6 due to the extremely small order of magnitude of each iteration's perturbation (between 10^-8 and 10^-5)
        perturbation, iters, fool_label = deepfool(fold_model_rnn, example_input, eta=1e6)

        # save the ||r(x)|| / ||x|| value into the list defined in the beginning of the "fold" for loop
        robustness_values_rnn_fold.append(example_robustness(example_input, perturbation))

    del X_fold
    gc.collect()

    # to save the results of each fold
    save_pkl(robustness_values_rnn_fold, f"robustness/robustness_values_rnn_fold{f}.pkl")

### Considerations

Considering that the deepfool algorithm took a long time to run, we were only able to evaluate the robustness of the RNN-LSTM model.

## References

- Moosavi-Dezfooli, S.-M., Fawzi, A., Frossard, P., Polytechnique, E. and De Lausanne, F. (2016). DeepFool: a simple and accurate method to fool deep neural networks: https://openaccess.thecvf.com/content_cvpr_2016/papers/Moosavi-Dezfooli_DeepFool_A_Simple_CVPR_2016_paper.pdf.

- TensorFlow. (2023). Introduction to gradients and automatic differentiation | TensorFlow Core. [online] Available at: https://www.tensorflow.org/guide/autodiff.